In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import os
from datetime import timedelta,datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
script_dir = '../scripts'

In [5]:
def  process_rhpi():

    #read in rhpi data

    rhpi = pd.read_csv(f'{script_dir}/../data/rhpi.csv',encoding="utf-16",sep='\t')

    #subset data for los angeles

    la_rhpi = rhpi[rhpi['Region Name'] == "Los Angeles, CA"][['Month, Year of Date','Redfin HPI MoM']]

    #convert types, rename, and sort index
    la_rhpi['Month, Year of Date'] = pd.to_datetime(la_rhpi['Month, Year of Date'])
    la_rhpi.rename(columns={'Month, Year of Date':'Date'},inplace=True)
    la_rhpi.sort_values('Date',inplace=True)
    la_rhpi.set_index('Date',inplace=True)

    #fix formatting
    la_rhpi['Redfin HPI MoM'] = la_rhpi['Redfin HPI MoM'].apply(lambda x:float(str(x).replace('%','')))

    #make time series just the RHPI month over month
    la_rhpi = la_rhpi['Redfin HPI MoM']

    la_rhpi.dropna(inplace=True)

    original_la_rhpi = la_rhpi.copy()
    #get last date in time series
    last_date = la_rhpi.index[-1].strftime('%Y-%m-%d')


    return la_rhpi,original_la_rhpi,last_date

In [54]:
la_rhpi

,"Month, Year of Date",Redfin HPI MoM
0,August 2024,-0.09%
1,July 2024,0.31%
2,June 2024,0.55%
3,May 2024,0.25%
4,April 2024,0.76%
...,...,...
147,May 2012,1.57%
148,April 2012,1.72%
149,March 2012,NaN
150,February 2012,NaN


In [53]:
rhpi = pd.read_csv(f'{script_dir}/../data/rhpi.csv',encoding="utf-16",sep='\t')

#subset data for los angeles

la_rhpi = rhpi[rhpi['Region Name'] == "Los Angeles, CA"][['Month, Year of Date','Redfin HPI MoM']]

In [35]:
def process_redfin_city_data(last_date):

    #import housing related information by city

    info = pd.read_csv(f'{script_dir}/../data/city_market_tracker.tsv000.gz',sep='\t',compression='gzip')

    #take subset for los angeles and take mean of variables across different periods and cities to get one mean per period for LA
    la_info = info[(info['parent_metro_region'] == 'Los Angeles, CA') & (info['property_type'] == 'Single Family Residential')]
    features = la_info.groupby(['period_begin'])[['median_sale_price_mom','median_list_price_mom','median_ppsf_mom','median_list_ppsf_mom','homes_sold_mom','new_listings_mom','inventory_mom','sold_above_list_mom','price_drops_mom','median_dom_mom','months_of_supply_mom','avg_sale_to_list_mom']].mean()

    #make index datetime type and set date to match rhpi
    features.index = pd.to_datetime(features.index)
    features = features[(features.index >= '2012-04-01') & (features.index <= last_date)]

    return features

In [9]:
def process_mortage_data(last_date):

    mortgage = pd.read_csv(f'{script_dir}/../data/MORTGAGE30US.csv')

    #set date in datetime format as index
    mortgage['DATE'] = pd.to_datetime(mortgage['DATE'])
    mortgage.set_index('DATE', inplace=True)

    #fill missing values that are '.' values with previous value
    mortgage = mortgage.replace('.',pd.NA).ffill()

    mortgage['MORTGAGE30US'] = mortgage['MORTGAGE30US'].apply(lambda x:float(x))

    #convert weekly data to monthly data and make every datetime the beginning of the month
    mortgage = mortgage.resample('M').mean()
    mortgage.index = mortgage.index.to_period('M').to_timestamp('D')

    #select dates to match rhpi data
    mortgage = mortgage[(mortgage.index >= '2012-04-01') & (mortgage.index <= last_date)]

    mortgage = mortgage['MORTGAGE30US']

    return mortgage


In [10]:
def process_unemployment(last_date):

    #read in unemployment data
    unemp = pd.read_csv(f'{script_dir}/../data/LOSA106UR.csv')
    unemp.rename(columns={'LOSA106UR':'unemployment rate'},inplace=True)

    #change date variable to datetime and make it the index
    unemp['DATE'] = pd.to_datetime(unemp['DATE'])
    unemp.set_index('DATE',inplace=True)

    #subset series to match la rhpi data
    unemp = unemp[(unemp.index >= '2012-04-01') & (unemp.index <= last_date)]
    unemp = unemp['unemployment rate']

    return unemp

In [11]:
def normalize_series(series):

    mean, std = series.mean(),series.std()
    series = (series - mean)/std

    return series

In [12]:
la_rhpi,original_la_rhpi,last_date = process_rhpi()

/var/folders/2q/pjmqzbkj145d3k9gnr_4bmcw0000gn/T/ipykernel_27201/4243727859.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  la_rhpi['Month, Year of Date'] = pd.to_datetime(la_rhpi['Month, Year of Date'])


In [36]:
features = process_redfin_city_data(last_date)

In [52]:
la_rhpi

Date
2012-04-01    1.72
2012-05-01    1.57
2012-06-01    1.89
2012-07-01    0.87
2012-08-01    1.48
              ... 
2024-04-01    0.76
2024-05-01    0.25
2024-06-01    0.55
2024-07-01    0.31
2024-08-01   -0.09
Name: Redfin HPI MoM, Length: 149, dtype: float64

#### historical

In [47]:
output = pd.read_csv(f'{script_dir}/../output_data/tableau.csv')
output.reset_index(inplace=True)
output.drop_duplicates('Date',inplace=True)
output['Date'] = output['Date'].apply(lambda x:pd.to_datetime(x))

In [48]:
current_metrics = pd.read_csv(f'{script_dir}/../output_data/model_metrics.csv')
current_metrics = current_metrics[['Pred Date','Forecast Pred']]
current_metrics.rename(columns={'Pred Date':'Date'},inplace=True)
current_metrics['Date'] = pd.to_datetime(current_metrics['Date'],format='%Y-%m-%d')

In [51]:
output

,index,Date,Redfin HPI MoM,Type
0,0,2012-04-01,1.720000,Series
1,1,2012-05-01,1.570000,Series
2,2,2012-06-01,1.890000,Series
3,3,2012-07-01,0.870000,Series
4,4,2012-08-01,1.480000,Series
...,...,...,...,...
147,147,2024-07-01,0.310000,Series
148,148,2024-08-01,-0.090000,Series
150,150,2024-09-01,0.768974,Forecast
151,151,2024-10-01,0.746742,Forecast


In [55]:
current_metrics

,Date,Forecast Pred
0,2024-06-01,0.583043
1,2024-07-01,0.953691
2,2024-08-01,0.056024
3,2024-09-01,0.768974


In [49]:
current_metrics.merge(output[['Date','Redfin HPI MoM']],on='Date',how='inner')

,Date,Forecast Pred,Redfin HPI MoM
0,2024-06-01,0.583043,0.550000
1,2024-07-01,0.953691,0.310000
2,2024-08-01,0.056024,-0.090000
3,2024-09-01,0.768974,0.768974
